In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
# Load data and print its shape

# Train data
train = pd.read_csv("data/train.csv")

# Test data 
test = pd.read_csv("data/test.csv")

# Print shape
print(f"train: {train.shape}")
print(f"test: {test.shape}")

train: (100000, 16)
test: (25000, 16)


In [3]:
# Drop unnecessary columns
train = train.drop(['message_id', 'reply_to_id', 'text'], axis=1)
test = test.drop(['message_id', 'reply_to_id', 'text'], axis=1)